<center><h1>Evaluating Model Performance</h1></center>

## 1. Testing quality of predictions

We now have a function that can predict the price for any living space we want to list as long as we know the number of people it can accommodate. The function we wrote represents a **machine learning model**, which means that it outputs a prediction based on the input to the model.

A simple way to test the quality of your model is to:

- split the dataset into 2 partitions:
    - the training set: contains the majority of the rows (75%)
    - the test set: contains the remaining minority of the rows (25%)

- use the rows in the training set to predict the price value for the rows in the test set
    - add new column named predicted_price to the test set
- compare the `predicted_price` values with the actual price values in the test set to see how accurate the predicted values were.

This validation process, where we use the training set to make predictions and the test set to predict values for, is known as **train/test validation**. Whenever you're performing machine learning, you want to perform validation of some kind to ensure that your machine learning model can make good predictions on new data. While train/test validation isn't perfect, we'll use it to understand the validation process, to select an error metric, and then we'll dive into a more robust validation process later in this course.

Let's modify the `predict_price` function to use only the rows in the training set, instead of the full dataset, to find the nearest neighbors, average the `price` values for those rows, and return the predicted price value. Then, we'll use this function to predict the price for just the rows in the test set. Once we have the predicted price values, we can compare with the true price values and start to understand the model's effectiveness in the next screen.

To start, we've gone ahead and assigned the first 75% of the rows in `dc_listings` to `train_df` and the last 25% of the rows to `test_df`. Here's a diagram explaining the split:

<img src="figs/train_test_split.png" width="800" heigth="600"/>


### Exercise

- Within the `predict_price` function, change the Dataframe that `temp_df` is assigned to. Change it from `dc_listings` to `train_df`, so only the training set is used.
- Use the Series method `apply` to pass all of the values in the `accommodates` column from `test_df` through the `predict_price` function.
- Assign the resulting Series object to the `predicted_price` column in `test_df`.

In [2]:
import numpy as np
import pandas as pd

In [3]:
dc_listings = pd.read_csv('dc_airbnb.csv')

In [ ]:
dc_listings['price'] = dc_listings['price'].str.replace('[\$,]', '').astype(float) 


In [11]:
train_df = dc_listings.iloc[:2792]
test_df = dc_listings.iloc[2792:]

In [18]:
def predict_price(new_listing):
    temp_df = dc_listings.copy()
    temp_df['distance'] = (temp_df['accommodates'] - new_listing).abs()
    return temp_df.sort_values(by='distance')['price'].head().mean()    

In [19]:
# exercise
def predict_price(new_listing):
    temp_df = train_df.copy()
    temp_df['distance'] = (temp_df['accommodates'] - new_listing).abs()
    return temp_df.sort_values(by='distance')['price'].head().mean()    

In [22]:
test_df['predicted_price'] = test_df['accommodates'].apply(predict_price)

C:\Users\RAFAEL~1.TOR\AppData\Local\Temp/ipykernel_6592/1789143025.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predicted_price'] = test_df['accommodates'].apply(predict_price)


## 2. Error Metrics

We now need a metric that quantifies how good the predictions were on the test set. This class of metrics is called an error metric. As the name suggests, an error metric quantifies how inaccurate our predictions were compared to the actual values. In our case, the error metric tells us how off our predicted price values were from the actual price values for the living spaces in the test dataset.

We could start by calculating the difference between each predicted and actual value and then averaging these differences. This is referred to as mean error but isn't an effective error metric for most cases. Mean error treats a positive difference differently than a negative difference, but we're really interested in how far off the prediction is in either the positive or negative direction. If the true price was 200 dollars and the model predicted 210 or 190 it's off by 10 dollars either way.

We can instead use the mean absolute error, where we compute the absolute value of each error before we average all the errors.

$$MAE = \frac{1}{n} \sum_{k=1}^{n} |actual_1 - predicted_1| + \cdots + |actual_n - predicted_n|$$

### Exercise
- Use `numpy.absolute()` to calculate the mean absolute error between `predicted_price` and `price`.
- Assign the MAE to `mae`.

In [26]:
mae = (test_df['predicted_price'] - test_df['price']).abs().mean()
print(mae)

56.29001074113876


## 3. Mean Squared Error

For many prediction tasks, we want to penalize predicted values that are further away from the actual value far more than those closer to the actual value.

We can instead take the mean of the squared error values, which is called the **mean squared error** or MSE for short. The MSE makes the gap between the predicted and actual values more clear. A prediction that's off by 100 dollars will have an error (of 10,000) that's 100 times more than a prediction that's off by only 10 dollars (which will have an error of 100).

Here's the formula for MSE:

$$MSE = \frac{1}{n} \sum_{k=1}^{n} (actual_1 - predicted_1)^2 + \cdots + (actual_n - predicted_n)^2$$

where `n` represents the number of rows in the test set. Let's calculate the MSE value for the predictions we made on the test set.

### Exercise
- Calculate the MSE value between the `predicted_price` and price columns and assign to `mse`.

In [28]:
mse = np.power(test_df['predicted_price']- test_df['price'], 2).mean()
print(mse)

18646.525370569325


## 4. Training another model
The model we trained achieved a mean squared error of around **18646.5**. Is this a high or a low mean squared error value? What does this tell us about the quality of the predictions and the model? By itself, the mean squared error value for a single model isn't all that useful.

The units of mean squared error in our case is dollars squared (not dollars), which makes it hard to reason about intuitively as well. We can, however, train another model and then compare the mean squared error values to see which model performs better on a relative basis. Recall that a low error metric means that the gap between the predicted list price and actual list price values is low while a high error metric means the gap is high.

Let's train another model, this time using the `bathrooms` column, and compare MSE values.

### Exercise

- Modify the `predict_price` function to the right to use the `bathrooms` column instead of the `accommodates` column to make predictions.
- Apply the function to test_df and assign the resulting Series object containing the predicted price values to the predicted_price column in test_df.
- Calculate the squared error between the price and predicted_price columns in test_df and assign the resulting Series object to the squared_error column in test_df.
- Calculate the mean of the squared_error column in test_df and assign to mse.
- Use the print function or the variables inspector to display the MSE value.

In [33]:
# exercise
def predict_price(new_listing):
    temp_df = train_df.copy()
    temp_df['distance'] = (temp_df['bathrooms'] - new_listing).abs()
    return temp_df.sort_values(by='distance')['price'].head().mean()    

In [34]:
test_df['predicted_price'] = test_df['bathrooms'].apply(predict_price)

C:\Users\RAFAEL~1.TOR\AppData\Local\Temp/ipykernel_6592/1150879193.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predicted_price'] = test_df['bathrooms'].apply(predict_price)


In [40]:
test_df['squared_error'] = (test_df['price'] - test_df['predicted_price'])**2
mse = test_df['squared_error'].mean()
print(mse)

18405.444081632548


C:\Users\RAFAEL~1.TOR\AppData\Local\Temp/ipykernel_6592/2349769556.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['squared_error'] = (test_df['price'] - test_df['predicted_price'])**2


## 5. Root Mean Squared Error
While comparing MSE values helps us identify which model performs better on a relative basis, it doesn't help us understand if the performance is good enough in general. This is because the units of the MSE metric are squared (in this case, dollars squared). An MSE value of 16377.5 dollars squared doesn't give us an intuitive sense of how far off the model's predictions are from the true price value in dollars.

**Root mean squared error** is an error metric whose units are the base unit (in our case, dollars). RMSE for short, this error metric is calculated by taking the square root of the MSE value:

$$RMSE = \sqrt{MSE}$$

Since the RMSE value uses the same units as the target column, we can understand how far off in real dollars we can expect the model to perform.

Let's calculate the RMSE value of the model we trained using the `bathrooms
` column.

### Exercise

Calculate the RMSE value of the model we trained using the bathrooms column and assign it to `rmse`.

In [41]:
rmse = np.sqrt(mse)
print(rmse)

135.6666653295221


## 6. Comparing MAE and RMSE
The model achieved an RMSE value of approximately 135.6, which implies that we should expect for the model to be off by 135.6 dollars on average for the predicted price values. Given that most of the living spaces are listed at just a few hundred dollars, we need to reduce this error as much as possible to improve the model's usefulness.

We discussed a few different error metrics we can use to understand a model's performance. As we mentioned earlier, these individual error metrics are helpful for comparing models. To better understand a specific model, we can compare multiple error metrics for the same model. This requires a better understanding of the mathematical properties of the error metrics.

If you look at the equation for MAE:

$$MAE = \frac{1}{n} \sum_{k=1}^{n} |actual_1 - predicted_1| + \cdots + |actual_n - predicted_n|$$


you'll notice that that the differences between predicted and actual values grow linearly. A prediction that's off by 10 dollars has a 10 times higher error than a prediction that's off by 1 dollar. If you look at the equation for RMSE, however:

$$RMAE = \sqrt{\frac{\sum_{k=1}^{n} (actual_1 - predicted_1)^2 + \cdots + (actual_n - predicted_n)^2}{n}}$$ 


you'll notice that each error is squared before the square root of the sum of all the errors is taken. This means that the individual errors grows quadratically and has a different effect on the final RMSE value.

Let's look at an example using different data entirely. We've created 2 Series objects containing 2 sets of errors and assigned to `errors_one` and `errors_two`.

### Exercise

- Calculate the MAE for `errors_one` and assign to `mae_one`.
- Calculate the RMSE for `errors_one` and assign to `rmse_one`.
- Calculate the MAE for `errors_two` and assign to `mae_two`.
- Calculate the RMSE for `errors_two` and assign to `rmse_two`.

```python
errors_one = pd.Series([5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10])
errors_two = pd.Series([5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 1000])
```

In [43]:
errors_one = pd.Series([5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10])
errors_two = pd.Series([5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 1000])

mae_one = errors_one.mean()
rmse_one = np.sqrt(np.power(errors_one, 2).mean())

mae_two = errors_two.mean()
rmse_two = np.sqrt(np.power(errors_two, 2).mean())